In [3]:
import os

format = """#!/bin/bash
#SBATCH --job-name=SPNet_{dataset}
#SBATCH --partition=kisski
#SBATCH -G A100:1
#SBATCH --cpus-per-task=16
#SBATCH --mem=20G
#SBATCH --time=2-00:00:00
#SBATCH --output=run_%x_%j.out
#SBATCH --error=run_%x_%j.err
#SBATCH --mail-type=FAIL          # Email on start, end, failure
#SBATCH --mail-user=hao.huang@tib.eu   # <-- Replace with your real email


echo "===== JOB STARTED ====="
echo "Hostname: $(hostname)"
echo "Date:     $(date)"
echo "User:     $USER"


# Load environment
module load miniforge3
module load gcc/13.2.0
module load cuda/11.8.0
# module load gcc/13.2.0
# module load cuda/12.6.2

# Set up conda
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate /mnt/vast-kisski/projects/kisski-tib-activecl/cenv

# Diagnostics
echo "Which python: $(which python)"
python -c "import torch; print('Torch:', torch.__version__, '| CUDA:', torch.cuda.is_available())"

# Navigate to project directory
cd /mnt/vast-kisski/projects/kisski-tib-deepci/SPNet-main_hao

# Run scripts
echo "--- Running: main_compare_balu.py {dataset} ---"
python3 -u main_compare_balu.py --dataset {dataset} --missing_p 0.0
python3 -u main_compare_balu.py --dataset {dataset} --missing_p 0.1
python3 -u main_compare_balu.py --dataset {dataset} --missing_p 0.3


echo "===== JOB COMPLETED ====="
echo "Date: $(date)"
"""

# datasets = ['Syn', 'AMZS', 'BlogCatalog1', "Flickr", "Flickr1", "Youtube"] , 'AMZS_M=20'
# datasets = ['Syn_M=None', 'BlogCatalog1_M=20', 'Flickr1_M=20', 'Flickr_M=20', 'AMZS_M=20']
datasets = ['Syn_M=None_SimRel=1_Rel=4',  'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4']
datasets += ['BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']
miss_patterns = ['MCAR'] #'MAR',
slurm_dir = 'u16914_slurm'

missing_ps = [0.0,0.1,0.3]

# Create the slurm directory if it doesn't exist
os.makedirs(slurm_dir, exist_ok=True)

# print(f"Generating SLURM files in '{slurm_dir}' and printing sbatch commands:\n")
print("cd /mnt/vast-kisski/projects/kisski-tib-deepci/SPNet-main_hao")
# Generate and save the slurm files and print commands
for miss_pattern in miss_patterns:
    for dataset in datasets:
        dataset += '_'+miss_pattern
        # Fill the format string with the current dataset name
        slurm_content = format.format(dataset=dataset)

        # Define the output filename
        filename = os.path.join(slurm_dir, f"{dataset}.slurm")

        # Write the content to the file
        with open(filename, "w") as f:
            f.write(slurm_content)

        # print(f"Generated {filename}")

        sbatch_command = f"sbatch {filename}"
        print(sbatch_command)


cd /mnt/vast-kisski/projects/kisski-tib-deepci/SPNet-main_hao
sbatch u16914_slurm/Syn_M=None_SimRel=1_Rel=4_MCAR.slurm
sbatch u16914_slurm/BlogCatalog1_M=20_SimRel=1_Rel=4_MCAR.slurm
sbatch u16914_slurm/Flickr1_M=20_SimRel=1_Rel=4_MCAR.slurm
sbatch u16914_slurm/Youtube_M=20_SimRel=1_Rel=4_MCAR.slurm
sbatch u16914_slurm/BlogCatalog1_M=20_SimRel=0_Rel=1_MCAR.slurm
sbatch u16914_slurm/Flickr1_M=20_SimRel=0_Rel=1_MCAR.slurm
